In [4]:
import pandas as pd

In [24]:
oos = pd.read_csv('samples.txt', sep = '\n', names=['sentences'])
triggers = pd.read_csv('triggers.txt', sep = '\n', names=['sentences'])
distractors_tbl = pd.read_csv('neg_distractors.txt', sep = '\n', names=['sentences'])
context = pd.read_csv('weather_app_sample_convos.csv')

In [33]:
def oos_context_gen(oos, triggers, context, distractors = True, len_oos = 20): 
    "Generates one random string of input for the weather app using len_oos OOS lines"
    
    oos_samp = oos.sample(30)
    context_samp = context.sample(1)
    triggers_samp = triggers.sample(1)
        
    oos_text = ' '.join(oos_samp['sentences'].values)
    context_text, label = context_samp['Context'].values[0], context_samp['Location'].values[0]
    trigger = triggers_samp['sentences'].values[0]
    
    if distractors == True:
        distractors_samp = distractors_tbl.sample(1)
        distractors_text = distractors_samp['sentences'].values[0]
        return ' '.join([oos_text, context_text, distractors_text, trigger]), label
    
    else: 
        return ' '.join([oos_text, context_text, trigger]), label
    

In [37]:
input_samples = pd.DataFrame(columns = ['sentence', 'label'])
for i in range(5000): 
    sent, lab = oos_context_gen(oos, triggers, context)
    input_samples = input_samples.append({'sentence' : sent, 'label' : lab}, ignore_index = True)


In [39]:
input_samples.to_csv('input_samples.csv')